In [1]:
pwd

'/Users/netcan/Workspace/Programming/FCEmulator/src/tools'

In [179]:
import InstructionReference

In [180]:
ret = InstructionReference.get_inst_ref()

In [185]:
import pandas as pd
df = pd.DataFrame(ret)

In [186]:
df['addressingMode'] = df['addressingMode'].str.replace('Implied', 'Implicit'
                                ).str.replace('Zero\s+Page', 'ZeroPage'
                                ).str.replace('ZeroPage,X', 'ZeroPageX'
                                ).str.replace('ZeroPage,Y', 'ZeroPageY'
                                ).str.replace('Absolute,X', 'AbsoluteX'
                                ).str.replace('Absolute,Y', 'AbsoluteY'
                                ).str.replace('\(Indirect,X\)', 'IndexIndirect'
                                ).str.replace('\(Indirect\),Y', 'IndirectIndex')
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,$69,2,2,0
1,ADC,ZeroPage,$65,2,3,0
2,ADC,ZeroPageX,$75,2,4,0
3,ADC,Absolute,$6D,3,4,0
4,ADC,AbsoluteX,$7D,3,4 (+1 if page crossed),0


In [187]:
df['code'] = df['code'].str.replace('$', '0x')
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4 (+1 if page crossed),0


In [188]:
df.loc[df['cycles'].str.contains('\+1'), 'extraCycles'] = 1
df.loc[df['cycles'].str.contains('\+2'), 'extraCycles'] = 2
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4 (+1 if page crossed),1


In [190]:
df['cycles'] = df['cycles'].str.replace('[\n\s]', ''
                            ).str.replace('(\d+).*', lambda s: s.group(1))
df.head()

,name,addressingMode,code,bytes,cycles,extraCycles
0,ADC,Immediate,0x69,2,2,0
1,ADC,ZeroPage,0x65,2,3,0
2,ADC,ZeroPageX,0x75,2,4,0
3,ADC,Absolute,0x6D,3,4,0
4,ADC,AbsoluteX,0x7D,3,4,1
